## Requirements


#### Directory adjustment


In [1]:
from pathlib import Path
import sys
import os

# Back to main folder
path = os.path.dirname(os.getcwd()) + "/"
os.chdir(path)
sys.path.append(path)


#### Data Processing


In [2]:
# ETL
import numpy as np
import pandas as pd


#### Natural language processing


In [3]:
from gensim.models import KeyedVectors
from gensim import models
import gensim


/home/kunumi/miniconda3/envs/hate-seepch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Models
[Check](https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)

In [4]:
# ML preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split

# Deep learnig model
from keras.wrappers.scikit_learn import KerasClassifier
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import LSTM


2022-10-06 14:15:21.612067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-06 14:15:21.903980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 14:15:21.903995: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-06 14:15:21.935634: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-06 14:15:22.721741: W tensorflow/stream_executor/platform/de

In [5]:
# Load pretrained word embedding
from email import header


# word_vectors = KeyedVectors.load_word2vec_format(
#     "data/pretrained-glove/glove.twitter.27B.200d.txt", binary=False, no_header=True
# )

word_vectors = KeyedVectors.load_word2vec_format(
    "data/pretrained-glove/glove_s300.txt", binary=False
)


## Split dataset


In [6]:
# Get data
df = pd.read_csv("data/corpus/augmented_corpus_fortuna.csv")
df.head(2)


,text_nonstop,text_lemma,text,length_text_nonstop,length_text_lemma,length_text,label,count_word_text_nonstop,count_word_text_lemma,count_word_text,...,pron,adp,aux,cconj,num,space,intj,sym,punct,part
0,cara vive outro mundo nao mundo real refugiado...,caro viver outro mundo nao mundo real refugiad...,nomeusuario o cara vive em outro mundo nao no ...,85,82,124,1,19,19,20,...,0,0,0,0,0,0,0,0,0,0
1,incompetentes nao cuidam povo brasileiro pouco...,incompetente nao cuidar povo brasileiro pouco ...,nomeusuario estes incompetentes nao cuidam nem...,69,66,108,0,20,20,20,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Set target and features
target = "label"
features = "text_nonstop"
count = f"length_{features}"

# Break apart dataset
X = df[features].values.astype("U")
y = df[target]

# Split train abd test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Set k-fold criteria
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

# Corpus considerations
corpus = X
longest_text = df[count].max()


## Teste 1

In [8]:
# from numpy import array
# from numpy import asarray
# from numpy import zeros
# from keras.preprocessing.text import Tokenizer
# from keras_preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import Embedding

In [9]:
# # prepare tokenizer
# t = Tokenizer()
# t.fit_on_texts(corpus)
# vocab_size = len(t.word_index) + 1
# # integer encode the documents
# encoded_docs = t.texts_to_sequences(corpus)

In [10]:
# # pad documents to a max length of 4 words
# max_length = 4
# padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [11]:
# # load the whole embedding into memory
# embeddings_index = dict()
# f = open('data/pretrained-glove/glove.twitter.27B.200d.txt')
# for line in f:
# 	values = line.split()
# 	word = values[0]
# 	coefs = asarray(values[1:], dtype='float32')
# 	embeddings_index[word] = coefs
# f.close()
# print('Loaded %s word vectors.' % len(embeddings_index))

In [12]:
# # create a weight matrix for words in training docs
# embedding_matrix = zeros((vocab_size, 200))
# for word, i in t.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector


In [13]:
# # define model
# model = Sequential()
# e = Embedding(
#     vocab_size, 200, weights=[embedding_matrix], input_length=4, trainable=False
# )
# model.add(e)
# model.add(Flatten())
# model.add(Dense(1, activation="sigmoid"))
# # compile the model
# model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# # summarize the model
# print(model.summary())


In [14]:
# # fit the model
# model.fit(padded_docs, y, epochs=50, verbose=0)
# # evaluate the model
# loss, accuracy = model.evaluate(padded_docs, y, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

## Teste 2

In [15]:
class TokenizerTransformer(BaseEstimator, TransformerMixin, Tokenizer):
    def __init__(self, **tokenizer_params):
        Tokenizer.__init__(self, **tokenizer_params)

    def fit(self, X, y=None):
        self.fit_on_texts(X)
        return self

    def transform(self, X, y=None):
        X_transformed = self.texts_to_sequences(X)
        return X_transformed


class PadSequencesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, maxlen):
        self.maxlen = maxlen

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_padded = pad_sequences(X, maxlen=self.maxlen)
        return X_padded


In [16]:
def create_model(embedding_input_dim, embedding_output_dim, embedding_weights):
    model = Sequential()
    model.add(
        Embedding(
            input_dim=embedding_input_dim,
            output_dim=embedding_output_dim,
            weights=[embedding_weights],
            trainable=False,
            mask_zero=True,
        )
    )
    model.add(LSTM(200))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model


In [17]:
embedding_weights = np.vstack([
           np.zeros(word_vectors.vectors.shape[1]),
           word_vectors.vectors
])

In [18]:
vocab_size = len(word_vectors) + 1
EMBEDDING_DIM = word_vectors.vectors.shape[1]

In [19]:
my_tokenizer = TokenizerTransformer()

my_padder = PadSequencesTransformer(maxlen=longest_text)

my_model = KerasClassifier(
    build_fn=create_model,
    epochs=50,
    embedding_input_dim=vocab_size,
    embedding_output_dim=EMBEDDING_DIM,
    embedding_weights=embedding_weights,
    callbacks=[EarlyStopping(monitor="loss", patience=10)],
)

pipeline = Pipeline(
    [("tokenizer", my_tokenizer), ("padder", my_padder), ("model", my_model)]
)


/tmp/ipykernel_65825/1927001707.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  my_model = KerasClassifier(


In [20]:
pipeline.fit(X_train, y_train)
pd.DataFrame(
    classification_report(y_test, pipeline.predict(X_test), output_dict=True)
).T


2022-10-06 14:17:24.259128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 14:17:24.259383: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 14:17:24.259452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-06 14:17:24.259512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-06 14:17:24.259570: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/50
142/142 [==============================] - 18s 107ms/step - loss: 0.6117 - accuracy: 0.6951
Epoch 2/50
142/142 [==============================] - 15s 103ms/step - loss: 0.5545 - accuracy: 0.7220
Epoch 3/50
142/142 [==============================] - 15s 104ms/step - loss: 0.5041 - accuracy: 0.7551
Epoch 4/50
142/142 [==============================] - 15s 104ms/step - loss: 0.4404 - accuracy: 0.7963
Epoch 5/50
142/142 [==============================] - 15s 106ms/step - loss: 0.3548 - accuracy: 0.8455
Epoch 6/50
142/142 [==============================] - 15s 105ms/step - loss: 0.2358 - accuracy: 0.9074
Epoch 7/50
142/142 [==============================] - 15s 107ms/step - loss: 0.1372 - accuracy: 0.9559
Epoch 8/50
142/142 [==============================] - 15s 106ms/step - loss: 0.0846 - accuracy: 0.9777
Epoch 9/50
142/142 [==============================] - 15s 106ms/step - loss: 0.0598 - accuracy: 0.9854
Epoch 10/50
142/142 [==============================] - 15s 107ms/step - l

,precision,recall,f1-score,support
0,0.760000,0.806950,0.782772,777.000000
1,0.514563,0.445378,0.477477,357.000000
accuracy,0.693122,0.693122,0.693122,0.693122
macro avg,0.637282,0.626164,0.630125,1134.000000
weighted avg,0.682733,0.693122,0.686660,1134.000000
